# Basic EDA and Visualization

1. Understanding the datasets - Basic infos
2. Data Quality Checks
    1. Missing Value
    2. Outlier Check
    3. Duplicate Check
    4. Dataset column type checks
3. Univariate, BiVariate, MultiVariate Analysis and Visualization

In [2]:
#loading the required packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import plotnine as p9 
import seaborn as sns
from scipy import stats
from mizani.formatters import percent_format

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline 


In [6]:
#Fetching the data
raw_data = pd.read_csv('./data/tesco-dataset/train.csv')
print('The shape of the dataset :' + str(raw_data.shape))
raw_data.head()

The shape of the dataset :(320, 16)


,location_id,crime_rate,proportion_flats,proportion_nonretail,new_store,commercial_property,household_size,proportion_newbuilds,public_transport_dist,transport_availability,property_value,school_proximity,competitor_density,household_affluency,normalised_sales,county
0,464,17.600541,0.0,18.10,no,NaN,2.926,29.0,2.9084,All transport options,666,20.2,368.74,4.5325,-0.399933,c_40
1,504,0.603556,20.0,3.97,no,14.85,4.520,10.6,2.1398,Average transport options,264,13.0,388.37,1.8150,2.216308,c_80
2,295,0.606810,0.0,6.20,no,7.70,2.981,31.9,3.6715,Many transport options,307,17.4,378.35,2.9125,0.166920,c_53
3,187,0.012385,55.0,2.25,no,1.95,3.453,68.1,7.3073,No transport options,300,15.3,394.72,2.0575,-0.083804,c_65
4,193,0.016182,100.0,1.32,no,3.05,3.816,59.5,8.3248,Average transport options,256,15.1,392.90,0.9875,0.962693,c_97


## Checking dtype

In [10]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   location_id             320 non-null    int64  
 1   crime_rate              320 non-null    float64
 2   proportion_flats        320 non-null    float64
 3   proportion_nonretail    320 non-null    float64
 4   new_store               320 non-null    object 
 5   commercial_property     291 non-null    float64
 6   household_size          320 non-null    float64
 7   proportion_newbuilds    320 non-null    float64
 8   public_transport_dist   320 non-null    float64
 9   transport_availability  320 non-null    object 
 10  property_value          320 non-null    int64  
 11  school_proximity        257 non-null    float64
 12  competitor_density      320 non-null    float64
 13  household_affluency     320 non-null    float64
 14  normalised_sales        320 non-null    fl

## Categorical data


In [11]:
raw_data["new_store"].value_counts()

new_store
no     301
yes     19
Name: count, dtype: int64

### Comments on new_store column:  
1. This has only 2 values.
2. Should be converted into boolean column

In [12]:
raw_data["transport_availability"].value_counts()

transport_availability
All transport options        84
Average transport options    72
Few transport options        69
No transport options         53
Many transport options       42
Name: count, dtype: int64

### Comments on transport_availability column:  
1. Looks like ordinal column. 
2. Should be converted into an ordered column in this order
    1. All transport options > Many transport options > Average transport options > Few transport options > No transport options.

In [40]:
raw_data["county"].value_counts().tail(50)

county
c_128    2
c_28     2
c_93     2
c_92     2
c_37     2
c_34     2
c_30     2
c_116    2
c_40     2
c_43     2
c_90     2
c_31     2
c_51     2
c_47     2
c_98     2
c_36     2
c_95     2
c_81     2
c_85     2
c_75     2
c_133    1
c_112    1
c_21     1
c_32     1
c_123    1
c_91     1
c_122    1
c_0      1
c_110    1
c_79     1
c_114    1
c_15     1
c_33     1
c_20     1
c_137    1
c_97     1
c_22     1
c_16     1
c_89     1
c_103    1
c_138    1
c_96     1
c_100    1
c_144    1
c_9      1
c_111    1
c_107    1
c_7      1
c_139    1
c_99     1
Name: count, dtype: int64

#### Comments on county column:
1. High cardinality
2. Considering only top 10 columns and remaining columns as 'others' - This is an option
3. Have to check correlation with sales to gain more clarity

#### Further analysis on county column

#### Checking if there is correlation between frequency and sales in the county

In [37]:
freq=raw_data["county"].value_counts()

sales=(raw_data
 .groupby("county")
 .aggregate({"normalised_sales":"mean"})
 .reset_index()
)

merged_df=pd.merge(freq,sales,on="county" )

merged_df["count"].corr(merged_df["normalised_sales"])

-0.3244946825682793

* No correlation 

Considering only top 10 columns and remaining columns as 'others'

#### Approach for handling "county":

* Going with Target variable based grouping - high, medium, low - with respect to sales


## Numerical columns

In [48]:
# Fetching numerical columns
numerical_columns=raw_data.select_dtypes(include=['int64', 'float64']).columns.to_list()

In [ ]:
#Checking the distribution for numerical variables 
for var in numerical_cols:
    print(
        (
             p9.ggplot(raw_data)
            +p9.aes(x=var)
            +p9.geom_bar()
            +p9.theme(figure_size=(6,3))
            +p9.facet_wrap("matched")
            + p9.facet_wrap("~matched", scales='free_y')
        )
    )

## Missing value checks

In [8]:
(
raw_data
.isna()
.sum()
)

location_id                0
crime_rate                 0
proportion_flats           0
proportion_nonretail       0
new_store                  0
commercial_property       29
household_size             0
proportion_newbuilds       0
public_transport_dist      0
transport_availability     0
property_value             0
school_proximity          63
competitor_density         0
household_affluency        0
normalised_sales           0
county                     0
dtype: int64

### Missing values in the follwing columns: 
1. commercial_property
2. school_proximity

# Numerical - Univariate

In [ ]:
for var in feature_cols:
    print(
        (
             p9.ggplot(raw_data)
            +p9.aes(x=var)
            +p9.geom_bar()
            +p9.theme(figure_size=(6,3))
            +p9.facet_wrap("matched")
            + p9.facet_wrap("~matched", scales='free_y')
        )
    )